In [5]:
import numpy as np
import math
from scipy.stats import t
import matplotlib.pyplot as plt

# Parameters 
m = 10
mst = 8
mtbc = 1

n_sim = 10
block_fractions = np.zeros(n_sim)
block_Z = np.zeros(n_sim)
mean_arr_times = np.zeros(n_sim)

for j in range(n_sim):
    n_custom = 10000
    n = 0

    interval_between_arrivals = np.random.exponential(scale = mtbc, size = n_custom)
    arrival_times = np.cumsum(interval_between_arrivals)
    mean_arr_times[j] = np.mean(interval_between_arrivals)

    service_in_use_times = np.zeros(m)

    n_cust_blocked = 0


    while(n < n_custom):

        available_services = np.where(arrival_times[n] >= service_in_use_times)[0]
        if (len(available_services)>0):
            i = available_services[0]
            service_time = np.random.exponential(scale=mst)
            service_in_use_times[i] = arrival_times[n] + service_time
        else:
            n_cust_blocked += 1
        n += 1

    fraction = n_cust_blocked / n_custom  * 100
    #print(f'fraction of blocks: {fraction}%')
    block_fractions[j] = fraction

# Exact solution
A = mst * mtbc
m_vect = np.arange(0,m+1)
denominator = np.sum(A**m_vect / [math.factorial(k) for k in m_vect])
B = A**m/math.factorial(m) / denominator

print(f'exact solution: {B*100}%')

# Confidence interval

theta_hat = np.sum(block_fractions)/n_sim
#print(theta_hat)
sigma_2 = (np.sum(block_fractions ** 2) - n_sim * theta_hat**2) / (n_sim -1)
  

dof = n_sim -1
alpha = 0.05
t_quant = t.ppf(1-alpha/2,dof)
CI = [theta_hat - np.sqrt(sigma_2)/np.sqrt(n)*t_quant, theta_hat + np.sqrt(sigma_2)/np.sqrt(n)*t_quant]

print(f'Confidence interval for standard method: {CI}')

# Control variate

c = - np.cov(block_fractions, mean_arr_times)[0, 1] / np.var(mean_arr_times)
Z = block_fractions + c*(mean_arr_times-mtbc)
Z_cv = np.sum(Z)/n_sim
Z_var = (np.sum(Z ** 2) - n_sim * Z_cv**2) / (n_sim -1)

# Confidence interval for control variate
 

dof = n_sim -1
alpha = 0.05
t_quant = t.ppf(1-alpha/2,dof)
CI_cv = [Z_cv - np.sqrt(Z_var)/np.sqrt(n)*t_quant, Z_cv + np.sqrt(Z_var)/np.sqrt(n)*t_quant]

print(f'Confidence interval for control variate: {CI_cv}')

print(f'Original variance: {sigma_2}, Control variate variance {Z_var}')

print(f"Amplitude of CI:\n Without CV: {CI[1]-CI[0]}\n With CV: {CI_cv[1]-CI_cv[0]}")

exact solution: 12.16610642529515%
Confidence interval for standard method: [11.985471512866374, 12.000528487133627]
Confidence interval for control variate: [12.035327555519471, 12.048473585488464]
Original variance: 0.1107566666666268, Control variate variance 0.08442745468659672
Amplitude of CI:
 Without CV: 0.015056974267253054
 With CV: 0.013146029968993389
